# IBM Applied Data Science Capstone

## Introduction

For the purposes of this final project, I have decided that I will explore restaurant locations in the state of New York; specifically the cities of Buffalo, Ithaca, and Albany. This will be done to gain better insights into restaurant investment opportunities. I will explore the types of cuisine and popularity and what each city should concentrate on when surveying for potential locations.

The question that will be attempted to be addressed is how we can predict how much popularity a new restaurant opening in each respective city can expect to get based on the type of food served. This will be analyzed based on how many "likes" a new restaurant opening in each city. I will use machine learning by comparing both linear and logistic regressions and compare which method produced better predictive capability.

## Data

In this project, I will specifically focus on restaurants within a 100 mile radius from coordinated provided by the Foursquare geolocator. The Foursquare API will likely provide more data and venues than necessary, so I will remove non-restaurant rows to focus the data. I will also analyze and compare the 'likes' data in order to draw the final conclusions. 

I will specifically be looking at the geographical coordinates of the three cities mentioned above. I will use the Foursquare API to obtain raw data, which will be focused on the specific name, latitudinal and longitudinal coordinates, category/type, and which respective city the data is referring to.